# Wildfires in the California: Getting the data

Author: Lena Berger

*Disclaimer: This project was done as part of a data science boot camp. All stakeholders mentioned are fictional. This study was not commissioned by any agency or stakeholder. The project makes use of fire data collected by Short (2017) and data provided by NOAA. Some of the findings presented in this document are based on a subsamples of these datasets and are not representative of and should not be generalized to the entire database without further due. Please contact the author previous to use or further distribution of the findings or analyses. The author does not take any responsibility or liability for consequences resulting from use of these analyses.*

The data is retrieved from Kaggle. The data sources are as follows: 

*Fire data*: Short, Karen C. 2017. Spatial wildfire occurrence data for the United States, 1992-2015 [FPAFOD20170508]. 4th Edition. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2013-0009.4 (Accessed through https://www.kaggle.com/rtatman/188-million-us-wildfires)

*Weather data*: NOAA. (Accessed through: https://www.kaggle.com/noaa/gsod?select=gsod2019; see also details on the licensing: https://www.data.gov/privacy-policy#data_policy, data labeled "Federal" at https://catalog.data.gov/dataset/global-surface-summary-of-the-day-gsod, July 23, 2020)

The fire data is an SQL database. The weather data is queried using Google's BigQuery API. When using this script make sure that you either have an environment that can access your Google Cloud authentication credentials or adjust the code to include your credentials. The output of this script is the fire data in csv format, the weather stations information in csv format, and the weather data for the years 1991 to 2015 as csv format (one file per year). 

## Preparation

In [1]:
# Get the packages
import pandas as pd
import sqlite3
import numpy as np
import seaborn as sns
from tqdm import tqdm
import mpu
import pandas as pd
import time
from google.cloud import bigquery

# Get the functions and dictionaries from the external file
exec(open("wildfires_functions_2020_07_26.py").read())

## Get the fire data
We get the fire data from the SQL database. The code assumes that the database is stored locally as sqlite. 

In [2]:
# Source: https://www.kaggle.com/rtatman/188-million-us-wildfires
# Years: 1992-2015
# The sql file is saved locally. 

In [3]:
# Connect to the database
sql_connect = sqlite3.connect('FPA_FOD_20170508.sqlite')
cursor = sql_connect.cursor()

In [4]:
# Query
query = "SELECT FOD_ID, FPA_ID, FIRE_CODE, FIRE_NAME, COMPLEX_NAME, FIRE_YEAR, DISCOVERY_DATE, DISCOVERY_DOY, DISCOVERY_TIME, STAT_CAUSE_CODE, STAT_CAUSE_DESCR, CONT_DATE, CONT_DOY, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE, LONGITUDE, OWNER_CODE, OWNER_DESCR, OWNER_DESCR, STATE, COUNTY, FIPS_CODE, FIPS_NAME FROM Fires"
results = cursor.execute(query).fetchall()
fires = pd.read_sql_query(query,sql_connect)

In [5]:
# Look at the data
fires.head()

,FOD_ID,FPA_ID,FIRE_CODE,FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,1,FS-1418826,BJ8K,FOUNTAIN,None,2005,2453403.5,33,1300,9.0,...,A,40.036944,-121.005833,5.0,USFS,USFS,CA,63,063,Plumas
1,2,FS-1418827,AAC0,PIGEON,None,2004,2453137.5,133,0845,1.0,...,A,38.933056,-120.404444,5.0,USFS,USFS,CA,61,061,Placer
2,3,FS-1418835,A32W,SLACK,None,2004,2453156.5,152,1921,5.0,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,STATE OR PRIVATE,CA,17,017,El Dorado
3,4,FS-1418845,None,DEER,None,2004,2453184.5,180,1600,1.0,...,A,38.559167,-119.913333,5.0,USFS,USFS,CA,3,003,Alpine
4,5,FS-1418847,None,STEVENOT,None,2004,2453184.5,180,1600,1.0,...,A,38.559167,-119.933056,5.0,USFS,USFS,CA,3,003,Alpine


In [6]:
# Save the data
# fires.to_csv('data/fire data 2020_06_25.csv', index=False)

## Get weather data
We will get the weather data for all US statsions for the years 1991 to 2015. We will query this data from Kaggle using Google's BigQuery API. The data for each year are stored in a separate file. 

In [7]:
# Source: https://www.kaggle.com/noaa/gsod?select=gsod2019
# Licensing: https://www.data.gov/privacy-policy#data_policy

In [8]:
# Let's start by getting a list of the weather stations
data = BigQueryHelper(active_project="bigquery-public-data", dataset_name="noaa_gsod")
query1 = """SELECT * FROM `bigquery-public-data.noaa_gsod.stations`
WHERE country IN ("US")"""
weather_stations = data.query_to_pandas_safe(query1, max_gb_scanned=1)

In [9]:
# Save the data
# weather_stations.to_csv('data/weather stations 2020_07_06.csv', index=False)

In [10]:
weather_stations.head()

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
0,690014,99999,C STN WHITE SANDS,US,NM,None,32.350,-106.367,+1224.0,19870101,19910611
1,690020,93218,JOLON HUNTER LIGGETT MIL RES,US,CA,KHGT,36.000,-121.233,+0317.0,19640715,19970401
2,690020,99999,JOLON HUNTER LIGGETT MIL RES,US,CA,KHGT,36.000,-121.233,+0317.0,20030702,20030801
3,690070,93217,FRITZSCHE AAF,US,CA,KOAR,36.683,-121.767,+0043.0,19600404,19930831
4,690110,99999,SELFRIDGE ANGB/TRAIN,US,MI,None,42.608,-82.835,+0177.0,19430614,19920507


In [11]:
# Drop irrelevant cases
weather_stations['begin'] = pd.to_datetime(weather_stations['begin'])
weather_stations['end'] = pd.to_datetime(weather_stations['end'])
weather_stations.dropna(subset = ["lat", "lon"], inplace = True)
weather_stations.drop(index = weather_stations[weather_stations["usaf"] == "999999"].index, inplace = True)
weather_stations.head()

,usaf,wban,name,country,state,call,lat,lon,elev,begin,end
0,690014,99999,C STN WHITE SANDS,US,NM,None,32.350,-106.367,+1224.0,1987-01-01,1991-06-11
1,690020,93218,JOLON HUNTER LIGGETT MIL RES,US,CA,KHGT,36.000,-121.233,+0317.0,1964-07-15,1997-04-01
2,690020,99999,JOLON HUNTER LIGGETT MIL RES,US,CA,KHGT,36.000,-121.233,+0317.0,2003-07-02,2003-08-01
3,690070,93217,FRITZSCHE AAF,US,CA,KOAR,36.683,-121.767,+0043.0,1960-04-04,1993-08-31
4,690110,99999,SELFRIDGE ANGB/TRAIN,US,MI,None,42.608,-82.835,+0177.0,1943-06-14,1992-05-07


In [12]:
# Extract the relevant stations
relevant_stations = weather_stations["usaf"]

In [13]:
# Get the data queried
years = range(1991, 2016)
for year in years: 
    data = BigQueryHelper(active_project = "bigquery-public-data", dataset_name = "noaa_gsod")
    query1 = """SELECT fire_year.stn, fire_year.year, fire_year.mo, fire_year.da, 
            fire_year.temp, fire_year.wdsp, fire_year.mxpsd, fire_year.gust, fire_year.max, 
            fire_year.min, fire_year.prcp, fire_year.sndp, fire_year.fog, fire_year.rain_drizzle, 
            fire_year.snow_ice_pellets, fire_year.hail, fire_year.thunder, fire_year.flag_prcp
            FROM {table_1} fire_year
            WHERE fire_year.stn IN UNNEST({station}) 
            """.format(table_1 = ('bigquery-public-data.noaa_gsod.gsod' + str(year)),
                       station = list(relevant_stations)
                      )

    print(data.estimate_query_size(query1))
            
    df = data.query_to_pandas_safe(query1, max_gb_scanned = 1)
    # print(df)
    
    df.to_csv('data/weather stations data test {y} 2020_07_25.csv'.format(y = year), index=False)

0.23536780755966902
0.23239799495786428
0.235555830411613
0.23786427173763514
0.23241228889673948
0.22986892145127058
0.23115433286875486
0.2305995961651206
0.23504323605448008
0.23361281864345074
0.24270772654563189
0.25598523672670126
0.2582026533782482
0.2707955799996853
0.316194468177855
0.2993541145697236
0.3057574350386858
0.32074865605682135
0.33270516991615295
0.34331436548382044
0.350805195979774
0.36713402438908815
0.3680956996977329
0.3779341662302613
0.38586911745369434
